In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM

In [2]:
df_path = '../data/clean_recipes.csv'
df = pd.read_csv(df_path, sep=';')

In [8]:
kgl1_df = pd.read_json('../data/kgl_ingredient_train.json')
kgl2_df = pd.read_json('../data/kgl_ingredient_test.json')

In [7]:
print(df.shape)
df.head()

(12351, 10)


,Recipe Name,Review Count,Recipe Photo,Author,Prepare Time,Cook Time,Total Time,Ingredients,Directions,RecipeID
0,Golden Crescent Rolls Recipe,304,https://images.media-allrecipes.com/userphotos...,Mike A.,25 m,15 m,3 h 10 m,"yeast,water,white sugar,salt,egg,butter,flour,...","Dissolve yeast in warm water.**Stir in sugar, ...",7000
1,Poppy Seed Bread with Glaze Recipe,137,https://images.media-allrecipes.com/userphotos...,Christina Jun,15 m,1 h,1 h 20 m,"flour,salt,baking powder,poppy,butter,vegetabl...",'Preheat oven to 350 degrees F (175 degrees C)...,7001
2,Applesauce Bread I Recipe,124,https://images.media-allrecipes.com/userphotos...,GAF55,10 m,1 h 20 m,1 h 30 m,"flour,egg,white sugar,vegetable oil,applesauce...",Preheat oven to 350 degrees F (175 degrees C)....,7003
3,Apple Raisin Bread Recipe,39,https://images.media-allrecipes.com/userphotos...,Helen Hanson,15 m,1 h,1 h 15 m,"flour,baking powder,baking soda,salt,cinnamon,...",Preheat oven to 350 degrees F (175 degrees C)....,7006
4,Buttermilk Oatmeal Bread Recipe,41,https://images.media-allrecipes.com/userphotos...,Helen Hanson,10 m,1 h,1 h 40 m,"oat,buttermilk,vegetable oil,egg,brown sugar,f...",Mix oats with buttermilk. Let stand for 1/2 h...,7007


In [12]:
ingr_series = df['Ingredients']
kgl1_series = kgl1_df['ingredients']
kgl2_series = kgl2_df['ingredients']

In [16]:
type(ingr_series[0]), type(kgl1_series[0]), type(kgl2_series[0])

(str, list, list)

In [17]:
ingr_series = ingr_series.apply(lambda x: x.split(','))

In [19]:
type(ingr_series[0]), type(kgl1_series[0]), type(kgl2_series[0])

(list, list, list)

In [20]:
print(len(ingr_series), len(kgl1_series), len(kgl2_series))

12351 39774 9944


In [21]:
ingr_list = ingr_series.append([kgl1_series, kgl2_series], ignore_index=True)
print(len(ingr_list))
ingr_list.head()

62069


0    [yeast, water, white sugar, salt, egg, butter,...
1    [flour, salt, baking powder, poppy, butter, ve...
2    [flour, egg, white sugar, vegetable oil, apple...
3    [flour, baking powder, baking soda, salt, cinn...
4    [oat, buttermilk, vegetable oil, egg, brown su...
dtype: object

In [41]:
ingredients = set(x for l in ingr_list for x in l)
ingredient_list = list(ingredients)

len(ingredient_list)

7800

In [43]:
ingr_int = {ingr:i for i,ingr in enumerate(ingredients)}
int_ingr = {i:ingr for i,ingr in enumerate(ingredients)}

In [46]:
ingr_int

ocolate sandwich cookies': 238,
 'recaito': 239,
 'Philadelphia Cream Cheese': 240,
 'Mission Corn Tortillas': 241,
 'beer': 242,
 'bow-tie pasta': 243,
 'Thai chili garlic sauce': 244,
 'mustard greens': 245,
 'ganache': 246,
 'cheese curds': 247,
 '2 1/2 fluid ounces cachaca': 248,
 'yellow bell pepper': 249,
 'reduced sodium chicken bouillon granules': 250,
 'amaretto': 251,
 'mochi': 252,
 'bone-in ribeye steak': 253,
 'boneless turkey breast': 254,
 '1/4 cup bamboo shoots': 255,
 '3 cups half-and-half': 256,
 'crumbled goat cheese': 257,
 'grill seasoning': 258,
 'crepes': 259,
 'whole wheat cheese tortellini': 260,
 'no-salt-added black beans': 261,
 'creamy gravy': 262,
 'chestnut spread': 263,
 'DRESSING': 264,
 'prepared coleslaw': 265,
 'roasted almonds': 266,
 'crumbled cheese': 267,
 'thin pizza crust': 268,
 'mochiko': 269,
 'sea cucumber': 270,
 'italian loaf': 271,
 'pita chips': 272,
 'chive': 273,
 ' split and toasted': 274,
 'sunflower kernels': 275,
 'digestive biscu

In [38]:
merged_df = ingr_list.apply(pd.Series)
merged_df = merged_df.rename(columns = lambda x: 'ingr_' + str(x))
merged_df.head(15)

,ingr_0,ingr_1,ingr_2,ingr_3,ingr_4,ingr_5,ingr_6,ingr_7,ingr_8,ingr_9,...,ingr_55,ingr_56,ingr_57,ingr_58,ingr_59,ingr_60,ingr_61,ingr_62,ingr_63,ingr_64
0,yeast,water,white sugar,salt,egg,butter,flour,butter,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,flour,salt,baking powder,poppy,butter,vegetable oil,egg,milk,white sugar,vanilla,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,flour,egg,white sugar,vegetable oil,applesauce,raisin,cinnamon,baking soda,baking powder,sour cream,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,flour,baking powder,baking soda,salt,cinnamon,nutmeg,brown sugar,oat,apple,walnut,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,oat,buttermilk,vegetable oil,egg,brown sugar,flour,baking powder,baking soda,salt,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,shortening,white sugar,salt,milk,egg,lemon,yeast,flour,fruit,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,shortening,water,brown sugar,yeast,water,white sugar,salt,bread,whole wheat,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,water,cottage cheese,vegetable oil,egg,flour,white sugar,baking soda,salt,yeast,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,margarine,white sugar,egg,banana,coffee,water,vanilla,flour,salt,baking powder,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,cornmeal,milk,egg,bell pepper,onion,garlic,salt,baking soda,white sugar,corn,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


* User starts to type in first ingredient
* we filter all rows that contain that given ingredient
* temp_df with filtered results

* user types in the second ingredient
* we filter all rows that contain that given ingredient from temp_df

* repeat

flour >> salt >> brown sugar

at this point - 